In [ ]:
from interviewer import Interviewer
import pyaudio
import ipywidgets as widgets
from IPython.display import display
import threading

interviewer = Interviewer()

# ========= Helper functions =========

def show_temp_text(message, duration=3):
    msg_widget = widgets.Text(value=message, disabled=True)
    display(msg_widget)
    threading.Timer(duration, lambda: msg_widget.close()).start()

# User text input handling
questions = [
    "What position are you interviewing for? ",
    "What level is this job? (Intern, entry-level, junior, etc.) ",
    "What company are you interviewing for? ",
    "What type of interview is this? ('B' for behavioral, 'T' for technical) ",
    "Any extra notes? "
]
user_answers = []
container = widgets.VBox()
display(container)

current_index = {'i': 0}

def ask_next_question():
    container.children = []

    if current_index['i'] >= len(questions):
        show_temp_text('Thank you for answering these questions. ', 5)
        interviewer.position = user_answers[0]
        interviewer.level = user_answers[1] 
        interviewer.company = user_answers[2]
        interviewer.type = user_answers[3]
        interviewer.notes = user_answers[4]

        starter_message = (
            f"Interview has started.\n"
            f"Applicant details: \n"
            f"Position: {interviewer.position}\n"
            f"Level: {interviewer.level}\n"
            f"Company: {interviewer.company}\n"
            f"Type: {interviewer.type}\n"
            f"Notes: {interviewer.notes}"
        )
        start_text = widgets.Textarea(value=starter_message, disabled=True, layout={'width': '1000px'})
        display(start_text)

        interviewer.conversation.clear()
        first_question = interviewer.ask_question()

        question_text.value = first_question
        display(question_text)

        return

    q_text = questions[current_index['i']]
    label = widgets.Label(q_text)
    input_box = widgets.Text(placeholder='Press [Enter] to submit answer')

    def on_submit(b):
        user_answers.append(input_box.value)
        current_index['i'] += 1
        ask_next_question()

    input_box.on_submit(on_submit)
    container.children = [label, input_box]

# ========= Start interview button =========

start_button = widgets.Button(
    description='Start Interview', disabled=False
)

def start_click(b):
    ask_next_question()
    interviewer.conversation.clear()
    start_button.disabled = True

display(start_button)

# ========= Recording control buttons =========

# i. Mic selection
def choose_mic():
    p = pyaudio.PyAudio()
    mic_options = []

    for i in range(p.get_device_count()):
        mic_name = p.get_device_info_by_host_api_device_index(0, i).get('name')
        mic_options.append((mic_name, i))

    p.terminate()
    return mic_options

mic_dropdown = widgets.Dropdown(
    options=choose_mic(),
    description='Choose mic',
    disabled=False
)

mic_container = widgets.VBox([mic_dropdown])
display(mic_container)

# ii. Start/stop controls
record_button = widgets.Button(
    description='Record', disabled=False, button_style='success', icon='microphone'
)
# record_text = widgets.Text(value='Started recording...', disabled=False)

def record_click(b):
    mic_index = mic_dropdown.value
    show_temp_text('Started recording...', 7)
    stop_button.disabled = False

    def run_start():
        interviewer.start_recording(mic_index)

    show_temp_text(f"Mic selected: {mic_dropdown.label}", 3)
    mic_container.children = []
    run_start()

stop_button = widgets.Button(
    description='End Response', disabled=True, button_style='warning', icon='stop'
)
stop_text = widgets.Text(value='Stopped recording. Loading AI response...', disabled=False)
# STOP THE THREAD
question_text = widgets.Textarea(
    value='', disabled=False, layout={'width':'1000px', 'height':'75px'}
)

def stop_click(b):
    # record_text.close()
    show_temp_text('Stopped recording. Loading AI response...', 3)
    # record_thread.stop()
    interviewer.stop_recording()
    next_question = interviewer.ask_question()
    
    question_text.value = next_question
    # with question_output:
    #     print(next_question)
    # display(question_output)

    # display(question_text)
    # question_text.close()

terminate_button = widgets.Button(
    description='Terminate', disabled=False, button_style='danger'
)

terminate_text = widgets.Text(value='Interview has ended.', disabled=False)

def terminate_click(b):
    interviewer.terminate()
    display(terminate_text)
    display(grade_button)

display(record_button, stop_button, terminate_button)

transcript_button = widgets.Button(
    description='Transcript [Toggle]', disabled=False, button_style='primary'
)

# ========= Display/evaluation buttons =========

grade_button = widgets.Button(
    description='Grade Me', disabled=False
)
grade_text = widgets.Textarea(value=interviewer.grade_interview(), disabled=False,layout={'width':'1000px', 'height':'50px'})
def grade_click(b):
    display(grade_text)

# transcript_text = widgets.Text(value='Transcript:', disabled=False)
transcript_output = widgets.Output()
transcript_visible = {'state':False}
transcript_container = widgets.VBox([transcript_button])
display(transcript_container)

def print_transcript():
    # display(transcript_text)
    return interviewer.get_conversation()

def toggle_transcript(b):
    # display(transcript_text)
    transcript_visible['state'] = not transcript_visible['state']
    transcript_output.clear_output()

    if transcript_visible['state']:
        with transcript_output:
            transcript_output.clear_output()
            print(print_transcript())
        transcript_container.children = [transcript_button, transcript_output]

    else:
        transcript_container.children = [transcript_button]

# on_click calls
start_button.on_click(start_click)
record_button.on_click(record_click)
stop_button.on_click(stop_click)
terminate_button.on_click(terminate_click)
transcript_button.on_click(toggle_transcript)
grade_button.on_click(grade_click)
# display(transcript_output)
